In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np
import requests
import json
import snscrape.modules.twitter as sntwitter

In [8]:
QUERY = 'natural gas'
LIMIT = 5

In [11]:
for tweet in sntwitter.TwitterSearchScraper(QUERY).get_items():
    print(tweet.content)

@Desi_FLA We don’t have any natural gas or oil reserves.
russia: Russia's weaponisation of natural gas could backfire by destroying demand for it, Energy News, ET EnergyWorld https://t.co/Fk0CqjLDsR @KCL_Artist @Riddlemethat14
@WillyLumpLump69 @RWApodcast Yea, they are scary loud and very common in every industrial complex that runs on natural gas -&gt; steam.
@Isegoria @bruce99340413 Certainly shifting from coal to natural gas is progress. But driving learning curves for cleaner energy sources will eventually bring about even cheaper energy that is much cleaner than natural gas as well.
@PLPSD @SecBlinken The irony in all of this, is the fact Europe pays $1 billion per day for Russian energy.

Italy's ENI has signed natural gas agreements with Algeria &amp; Egypt to replace Russian gas but it will take until summer of 2023 to do this.

Is this good or bad for the Russian economy? https://t.co/iJm67o0vhD
Why can’t Page us solar or wind power??? 
“Forty percent of Page's power comes fro

KeyboardInterrupt: 